In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import numpy as np

c:\Users\asami\Desktop\ING 5\NLP\Job-Similarity\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
data=pd.read_excel("data.xlsx",sheet_name=0)
data.head()

,JobID,Job,CompID,SkillsID,Competency/Skills
0,1,Data Analyst,C1,S1,Data Visualization Power BI
1,1,Data Analyst,C1,S2,Data Visualization Tableau
2,1,Data Analyst,C1,S3,Data Visualization Looker
3,1,Data Analyst,C1,S4,Data Visualization Matplotlib
4,1,Data Analyst,C2,S5,Data Preparation Pandas


In [38]:
skills=pd.DataFrame(list(data["Competency/Skills"].unique()),columns=["Skills"])
skills["SkillsID"]=list(data["SkillsID"].unique())
print(skills)

                               Skills SkillsID
0         Data Visualization Power BI       S1
1          Data Visualization Tableau       S2
2           Data Visualization Looker       S3
3       Data Visualization Matplotlib       S4
4             Data Preparation Pandas       S5
5              Data Preparation NumPy       S6
6                Data Preparation SQL       S7
7              Data Preparation Excel       S8
8       Machine Learning Scikit-learn       S9
9            Machine Learning XGBoost      S10
10          Machine Learning LightGBM      S11
11          Machine Learning CatBoost      S12
12             Statistical Analysis R      S13
13          Statistical Analysis SPSS      S14
14         Statistical Analysis Stata      S15
15           Deep Learning TensorFlow      S16
16              Deep Learning PyTorch      S17
17                Deep Learning Keras      S18
18            Model Deployment Docker      S19
19        Model Deployment Kubernetes      S20
20           

In [39]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [40]:
sentences = ["eat apple", "rabbit dead","mother loves fruits"]
embeddings = model.encode(sentences)
print(embeddings)

[[ 0.05198571  0.03726526  0.05079734 ... -0.01896067  0.0671699
   0.02756946]
 [ 0.047552   -0.01106954 -0.01533367 ...  0.0087118   0.01541569
  -0.01033785]
 [ 0.00875985  0.00015454  0.04707263 ... -0.00274574  0.11228129
  -0.01075795]]


In [5]:
print(embeddings[0].reshape(1,-1).shape)

(1, 384)


In [6]:
from sklearn.metrics.pairwise import cosine_similarity
similarity=cosine_similarity(embeddings[0].reshape(1,-1),embeddings[1].reshape(1,-1))
print(similarity)
similarity=cosine_similarity(embeddings[0].reshape(1,-1),embeddings[2].reshape(1,-1))
print(similarity)
similarity=cosine_similarity(embeddings[1].reshape(1,-1),embeddings[2].reshape(1,-1))
print(similarity)

[[0.07827369]]
[[0.43070382]]
[[0.0440757]]


In [41]:
import re
import nltk
from nltk.corpus import stopwords
nltk.download("stopwords")
stopFrench=stopwords.words("french")
def normalize(text):
  text=re.sub(r'http\S+', '', text)
  text=re.sub(r'@\w+', '', text)
  text=re.sub(r'[^a-zA-Z\s]', '', text)
  text=text.lower()
  words=text.split()
  words=[w for w in words if w not in stopFrench]
  text=" ".join(words)
  return text

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\asami\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [42]:
user_input="Son travail repose sur une vigilance constante et une capacité à distinguer les fausses alertes des véritables menaces. Lorsqu’un incident est confirmé, il enclenche les procédures de réponse : isolation des systèmes compromis, analyse des causes, et coordination avec les équipes techniques pour rétablir la sécurité. Chaque incident est une opportunité d’apprentissage, lui permettant d’affiner les règles de détection et de mieux préparer l’organisation aux attaques futures."
cleaned_user_input=normalize(user_input)
print(cleaned_user_input)

travail repose vigilance constante capacit distinguer fausses alertes vritables menaces lorsquun incident confirm enclenche procdures rponse isolation systmes compromis analyse causes coordination quipes techniques rtablir scurit chaque incident opportunit dapprentissage permettant daffiner rgles dtection mieux prparer lorganisation attaques futures


In [43]:
embedded_skills=model.encode(skills["Skills"].apply(normalize))
embedded_user=model.encode(cleaned_user_input)
similarities=[model.similarity(embedded_user,skills)[0][0].item() for skills in embedded_skills]
skills["Score"]=similarities

In [44]:
skills.head()

,Skills,SkillsID,Score
0,Data Visualization Power BI,S1,0.131679
1,Data Visualization Tableau,S2,0.113234
2,Data Visualization Looker,S3,0.114878
3,Data Visualization Matplotlib,S4,0.026105
4,Data Preparation Pandas,S5,0.123545


In [45]:
skills.sort_values(by=["Score"],ascending=False)

,Skills,SkillsID,Score
40,Threat Intelligence MITRE ATT&CK,S41,0.409731
39,Threat Intelligence MISP,S40,0.380052
41,Threat Intelligence ThreatConnect,S42,0.375090
36,Incident Response SIEM Tools,S37,0.360031
38,Incident Response Case Management,S39,0.300996
37,Incident Response Forensics Tools,S38,0.251620
14,Statistical Analysis Stata,S15,0.250966
12,Statistical Analysis R,S13,0.241878
13,Statistical Analysis SPSS,S14,0.240442
32,Security Monitoring QRadar,S33,0.229609


In [46]:
jobs=set(skills["Job"])
jobDf=pd.DataFrame(jobs,columns=["Job"])
jobDf.head()

KeyError: 'Job'

In [14]:
means=[]
for job in jobs:
    means.append(np.mean(skills[skills["Job"]==job]["Score"][0:3]))
jobDf["MeanScore"]=means
finalJob=jobDf.sort_values(by="MeanScore",ascending=False).reset_index(drop=True)
finalJob.head()

,Job,MeanScore
0,Cybersecurity Analyst,0.213775
1,SOC Analyst,0.213775
2,Pentester (Ethical Hacker),0.174229
3,Threat Intelligence Analyst,0.174229
4,Data Analyst,0.119931


In [15]:
print("Le metier qui vous correspond le plus est : ",finalJob.loc[0,"Job"])

Le metier qui vous correspond le plus est :  Cybersecurity Analyst
